In [2]:
import pandas as pd
import stanza
import requests

# ==============================================================================
# VAIHE 0: ALUSTUKSET (kuten aiemmin)
# ==============================================================================

# Stanza pipeline (ladataan vain kerran)
print("Alustetaan Stanza-putkea (voi kestää hetken)...")
nlp = stanza.Pipeline('fi', processors='tokenize,lemma', use_gpu=False, logging_level='WARN')
print("Stanza-putki valmis.")

# Stop-sanojen lataus ja karsinta
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-fi/master/stopwords-fi.txt"
response = requests.get(url)
finnish_stop_words = response.text.splitlines()
suodatus = {'ei', 'eikä', 'mutta', 'vaan', 'vaikka', 'jos', 'kuin'}
custom_stop_words = [word for word in finnish_stop_words if word not in suodatus]
custom_stop_words.append('placeholder') #tyhjien otsikoiden tilalle laitettu teksti

# Lemmatisointifunktio
def lemmatize_text(text):
    # Varmistetaan, että syöte on merkkijono
    if not isinstance(text, str):
        return ""
    doc = nlp(text)
    lemmatized_words = [word.lemma for sent in doc.sentences for word in sent.words]
    return ' '.join(lemmatized_words)

# ==============================================================================
# VAIHE 1: DATAN LATAUS JA VALMISTELU
# ==============================================================================

# Ladataan vanha, manuaalisesti leimattu aineisto mallin koulutusta varten
print("Ladataan prototyypin opetusdataa (arvostelut.csv)...")
df_train_prototype = pd.read_csv('arvostelut.csv', delimiter=';', quoting=0)

# Ladataan uusi, suuri ja leimaamaton aineisto
print("Ladataan uutta, suurta aineistoa (complete_reviews.csv)...")
df_new_reviews = pd.read_csv('complete_reviews.csv')

# --- Datan esikäsittely ---
print("Esikäsitellään dataa (lemmatisoidaan ja yhdistetään otsikot)...")

# Prototyypin opetusdata
df_train_prototype['otsikko'].fillna('', inplace=True) # Korvataan tyhjät otsikot
df_train_prototype['lemmatized_text'] = df_train_prototype.apply(lambda row: lemmatize_text(row['otsikko'] + ' ' + row['teksti']), axis=1)

# Uusi suuri data
df_new_reviews['otsikko'].fillna('', inplace=True) # Korvataan tyhjät otsikot
df_new_reviews['lemmatized_text'] = df_new_reviews.apply(lambda row: lemmatize_text(row['otsikko'] + ' ' + row['teksti']), axis=1)

print("Esikäsittely valmis.")

# ==============================================================================
# VAIHE 2: TIEDOSTO 1 - OHJELMAN LUOMAT TUNNEARVOT
# ==============================================================================

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Määritellään ja koulutetaan prototyyppimalli
print("Koulutetaan prototyyppimalli...")
X_train_proto = df_train_prototype['lemmatized_text']
y_train_proto = df_train_prototype['tunne']

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=custom_stop_words)),
    ('clf', LogisticRegression(class_weight='balanced')),
])

text_clf.fit(X_train_proto, y_train_proto)
print("Prototyyppimalli koulutettu.")

# Tehdään ennusteet koko uudelle aineistolle
print("Tehdään ennusteita uudelle aineistolle...")
X_new = df_new_reviews['lemmatized_text']
predicted_sentiments = text_clf.predict(X_new)

# Lisätään ennusteet uuteen DataFrameen ja tallennetaan
df_model_predictions = df_new_reviews.copy()
df_model_predictions['mallin_ennuste'] = predicted_sentiments

output_filename_1 = 'mallin_luomat_tunnearvot.csv'
df_model_predictions.to_csv(output_filename_1, index=False, encoding='utf-8-sig')
print(f"Tiedosto 1 tallennettu nimellä: {output_filename_1}")

# ==============================================================================
# VAIHE 3: TIEDOSTO 2 - TÄHTIEN PERUSTEELLA ANNETUT TUNNEARVOT
# ==============================================================================

def stars_to_sentiment(rating):
    """Muuttaa tähtiarvion (1-5) tunnesanaksi."""
    try:
        # Yritetään muuttaa arvo numeroksi. Gigantin '5.touko' muuttuu numeroksi 5.
        rating = int(float(rating)) 
        if rating <= 2:
            return 'negatiivinen'
        elif rating == 3:
            return 'neutraali'
        elif rating >= 4:
            return 'positiivinen'
    except (ValueError, TypeError):
        # Jos muunnos epäonnistuu (esim. tekstiä tai tyhjä), palautetaan N/A
        return 'N/A'

print("Luodaan tunnearvoja tähtien perusteella...")
# Luodaan uusi DataFrame, johon lisätään tähtipohjaiset tunnearvot
df_star_sentiments = df_new_reviews.copy()
df_star_sentiments['tahtien_tunne'] = df_star_sentiments['tahdet'].apply(stars_to_sentiment)

output_filename_2 = 'tahtien_perusteella_annetut_tunnearvot.csv'
df_star_sentiments.to_csv(output_filename_2, index=False, encoding='utf-8-sig')
print(f"Tiedosto 2 tallennettu nimellä: {output_filename_2}")

print("\nValmis! Voit nyt avata ja vertailla luotuja CSV-tiedostoja.")

Alustetaan Stanza-putkea (voi kestää hetken)...


2025-08-24 23:18:23 WARNING: Language fi package default expects mwt, which has been added


Stanza-putki valmis.
Ladataan prototyypin opetusdataa (arvostelut.csv)...
Ladataan uutta, suurta aineistoa (complete_reviews.csv)...
Esikäsitellään dataa (lemmatisoidaan ja yhdistetään otsikot)...


C:\Users\huvif\AppData\Local\Temp\ipykernel_25044\2139538703.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_prototype['otsikko'].fillna('', inplace=True) # Korvataan tyhjät otsikot
C:\Users\huvif\AppData\Local\Temp\ipykernel_25044\2139538703.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

Esikäsittely valmis.
Koulutetaan prototyyppimalli...
Prototyyppimalli koulutettu.
Tehdään ennusteita uudelle aineistolle...
Tiedosto 1 tallennettu nimellä: mallin_luomat_tunnearvot.csv
Luodaan tunnearvoja tähtien perusteella...
Tiedosto 2 tallennettu nimellä: tahtien_perusteella_annetut_tunnearvot.csv

Valmis! Voit nyt avata ja vertailla luotuja CSV-tiedostoja.
